# Chapter 2. Getting Started

### Excercise 2.1

In [3]:
def fib(n: Int): Int = {
  @annotation.tailrec
  def f(a: Int, b: Int, n: Int): Int =
    if (n > 1) f(b, a+b, n-1)
    else a

  f(0, 1, n)
}

defined function fib

In [4]:
fib(10)

res3: Int = 34

### Exercise 2.2

In [5]:
def isSorted[A](as: Array[A], ordered: (A,A) => Boolean): Boolean = {
  @annotation.tailrec
  def loop(n: Int): Boolean =
    if (n == as.length-1) true
    else if (ordered(as(n), as(n+1))) loop(n+1)
    else false

  loop(0)
}

defined function isSorted

### Exercise 2.3

In [6]:
def curry[A,B,C](f: (A,B) => C): A => B => C =
  a => b => f(a,b)

defined function curry

### Exercise 2.4

In [7]:
def uncurry[A,B,C](f: A => B => C): (A,B) => C =
  (a,b) => f(a)(b)

defined function uncurry

### Exercise 2.5

In [8]:
def compose[A,B,C](f: B => C, g: A => B): A => C =
  a => f(g(a))

defined function compose

# Chapter 3. Data Structures

## List

In [9]:
sealed trait List[+A]
case object Nil extends List[Nothing]
case class Cons[+A](head: A, tail: List[A]) extends List[A]


defined trait List
defined object Nil
defined class Cons

In [16]:
object List {
  def apply[A](as: A*): List[A] =
    if (as.isEmpty) Nil
    else Cons(as.head, apply(as.tail:_*))
}

List(1)
List("a")
List(1, 2, 3, 4)

defined object List
res15_1: List[Int] = Cons(1, Nil)
res15_2: List[String] = Cons("a", Nil)
res15_3: List[Int] = Cons(1, Cons(2, Cons(3, Cons(4, Nil))))

### Exercise 3.2

In [17]:
def tail[A](list: List[A]): List[A] =
  list match {
    case Nil => sys.error("empty list")
    case Cons(_, t) => t
  }

tail(res15_3)

defined function tail
res16_1: List[Int] = Cons(2, Cons(3, Cons(4, Nil)))

### Exercise 3.3

In [22]:
def setHead[A](list: List[A], h: A): List[A] =
  list match {
    case Nil => sys.error("empty list")
    case Cons(_, t) => Cons(h, t)
  }

setHead(res16_1, 11)

defined function setHead
res21_1: List[Int] = Cons(11, Cons(3, Cons(4, Nil)))

### Exercise 3.4

In [21]:
def drop[A](l: List[A], n: Int): List[A] =
  l match {
    case _ if n <= 0 => l
    case Nil => sys.error("empty list")
    case Cons(h, t) => drop(t, n-1)
  }

drop(res15_3, 2)

defined function drop
res20_1: List[Int] = Cons(3, Cons(4, Nil))

### Exercise 3.5

In [20]:
def dropWhile[A](l: List[A], f: A => Boolean): List[A] =
  l match {
    case Nil => Nil
    case Cons(h, t) =>
      if (f(h)) dropWhile(t, f)
      else l
  }

dropWhile(res15_3, (x: Int) => x < 3)

defined function dropWhile
res19_1: List[Int] = Cons(3, Cons(4, Nil))

### Exercise 3.6

In [23]:
def init[A](l: List[A]): List[A] =
  l match {
    case Nil => sys.error("empty list")
    case Cons(h, Nil) => Nil
    case Cons(h, t) => Cons(h, init(t))
  }

init(res15_1)
init(res15_2)
init(res15_3)

defined function init
res22_1: List[Int] = Nil
res22_2: List[String] = Nil
res22_3: List[Int] = Cons(1, Cons(2, Cons(3, Nil)))

### foldRight

In [24]:
def foldRight[A,B](as: List[A], z: B)(f: (A,B) => B): B =
  as match {
    case Nil => z
    case Cons(x, xs) => f(x, foldRight(xs, z)(f))
  }

defined function foldRight

Experiment to implement tail recursive `foldRight` (not an exercise)

In [26]:
def foldRight1[A,B](as: List[A], z: B)(f: (A,B) => B): B = {
  @annotation.tailrec
  def fn(acc: B => B, l: List[A]): B => B =
    l match {
      case Nil => acc
      case Cons(x, xs) => fn(f.curried(x) andThen acc, xs)
    }
  fn(x => x, as)(z)
}

defined function foldRight1

### Exercise 3.9

In [28]:
def length[A](as: List[A]): Int =
  foldRight(as, 0) { (_, n) => n+1 }

length(Nil)
length(res15_1)
length(res15_3)

defined function length
res27_1: Int = 0
res27_2: Int = 1
res27_3: Int = 4

implementation with `foldRight1`:

In [27]:
def length[A](as: List[A]): Int =
  foldRight1(as, 0) { (_, n) => n+1 }

length(Nil)
length(res15_1)
length(res15_3)

defined function length
res26_1: Int = 0
res26_2: Int = 1
res26_3: Int = 4

### Exercise 3.10: tail recursive `foldLeft`

In [29]:
@annotation.tailrec
def foldLeft[A,B](as: List[A], z: B)(f: (B,A) => B): B =
  as match {
    case Nil => z
    case Cons(x, xs) => foldLeft(xs,f(z,x))(f)
  }

defined function foldLeft

### Exercise 3.11: sum, product, length via foldLeft

In [30]:
def sum(as: List[Int]): Int = foldLeft[Int, Int](as, 0) { _ + _ }
def product(as: List[Double]): Double = foldLeft[Double, Double](as, 1.0) { _ * _ }
def length[A](as: List[A]): Int = foldLeft[A, Int](as, 0) { case (n,_) => n + 1 }


defined function sum
defined function product
defined function length

### Exercise 3.12: reverse

In [31]:
def reverse[A](as: List[A]): List[A] = {
  @annotation.tailrec
  def f(l: List[A], acc: List[A]): List[A] =
    l match {
      case Nil => acc
      case Cons(x, xs) => f(xs, Cons(x, acc))
    }
  f(as, Nil)
}

defined function reverse

via `fold`:

In [32]:
def reverse[A](as: List[A]): List[A] =
  foldLeft(as, Nil:List[A]) { (z,a) => Cons(a,z) }

defined function reverse

### Exercise 3.13: `foldLeft` via `foldRight`

In [33]:
def foldLeft[A,B](as: List[A], z: B)(f: (B,A) => B): B =
  foldRight(as, (b: B) => b)( (a,g) => b => g(f(b,a)) ) (z)

defined function foldLeft

other way around:

In [34]:
def foldRight[A,B](as: List[A], z: B)(f: (A,B) => B): B =
  foldLeft(as, (b: B) => b)( (g,a) => b => g(f(a,b)) ) (z)

defined function foldRight

### Exercise 3.14: append in terms of either `foldLeft` or `foldRight`

In [35]:
def append[A](xs: List[A], x: A): List[A] =
  foldRight(xs, Cons(x, Nil)) { Cons(_, _) }

def append1[A](xs1: List[A], xs2: List[A]): List[A] =
  foldRight(xs1, xs2) { Cons(_,_) }

defined function append
defined function append1

### Exercise 3.15: concatenate list of lists into single list

In [36]:
def concat[A](xs: List[List[A]]): List[A] =
  foldLeft(xs, Nil:List[A]) { foldRight(_, _) { Cons(_, _) } }

defined function concat

### Exercise 3.16 : transform a list of integers +1

In [37]:
def listP1(l: List[Int]): List[Int] =
  l match {
    case Nil => Nil
    case Cons(x, xs) => Cons(x+1, listP1(xs))
  }

defined function listP1

### Exercise 3.17 : transform a list of `Double`, turn every into string

In [38]:
def listDtoS(l: List[Double]): List[String] =
  l match {
    case Nil => Nil
    case Cons(x, xs) => Cons(x.toString, listDtoS(xs))
  }

defined function listDtoS

### Exercise 3.18 : `map`

In [39]:
def map[A,B](l: List[A])(f: A => B): List[B] =
  foldRight(l, Nil: List[B]) { (a,b) => Cons(f(a), b) }

defined function map

In [40]:
def map1[A,B](l: List[A])(f: A => B): List[B] =
  l match {
    case Nil => Nil
    case Cons(x, xs) => Cons(f(x), map(xs)(f))
  }

defined function map1

### Exercise 3.19 : `filter`

In [41]:
def filter[A](l: List[A])(f: A => Boolean): List[A] =
  l match {
    case Nil => Nil
    case Cons(x, xs) if f(x) => Cons(x, filter(xs)(f))
    case Cons(x, xs) => filter(xs)(f)
  }

defined function filter

### Exercise 3.20 : `flatMap`

In [42]:
def flatMap[A,B](l: List[A])(f: A => List[B]): List[B] =
  l match {
    case Nil => Nil
    case Cons(x, xs) => append1(f(x), flatMap(xs)(f))
  }

defined function flatMap

### Exercise 3.21 : `filter` via `flatMap`

In [43]:
def filter1[A](l: List[A])(f: A => Boolean): List[A] =
  flatMap(l) { x => if (f(x)) Cons(x, Nil) else Nil }

defined function filter1

### Exercise 3.23 : `zipWith`

In [44]:
def zipWith[A,B](l1: List[A], l2: List[A])(f: (A,A) => B): List[B] =
  (l1, l2) match {
    case (Cons(x1, xs1), Cons(x2, xs2)) => Cons(f(x1,x2), zipWith(xs1,xs2)(f))
    case (Nil, Nil) => Nil
    case _ => sys.error("different list sizes")
  }

defined function zipWith

### Exercise 3.24 : `hasSubsequence`

In [45]:
def hasSubsequence[A](full: List[A], sub: List[A]): Boolean =
  (full, sub) match {
    case (_, Nil) => true
    case (Nil, _) => false
    case (Cons(x1, f1), Cons(x2, f2)) =>
      if (x1 == x2) hasSubsequence(f1, f2)
      else hasSubsequence(f1, sub)
  }

defined function hasSubsequence

## Tree

In [46]:
sealed trait Tree[+A]
case class Leaf[A](value: A) extends Tree[A]
case class Branch[A](left: Tree[A], right: Tree[A]) extends Tree[A]

defined trait Tree
defined class Leaf
defined class Branch

### Exercise 3.25: `size` = the total number of leaves + branches

In [47]:
def size[A](tree: Tree[A]): Int =
  tree match {
    case Leaf(_) => 1
    case Branch(l, r) => 1 + size(l) + size(r)
  }

defined function size

### Exercise 3.26: `maximum` = the maximum element

In [48]:
def maximum(tree: Tree[Int]): Int =
  tree match {
    case Leaf(x) => x
    case Branch(l, r) => maximum(l) max maximum(r)
  }

defined function maximum

### Exercise 3.27: `depth`

In [49]:
def depth[A](tree: Tree[A]): Int =
  tree match {
    case Leaf(_) => 0
    case Branch(l,r) => 1 + (depth(l) max depth(r))
  }

defined function depth

### Exercise 3.28: `map`

In [50]:
def map[A,B](tree: Tree[A])(f: A => B): Tree[B] =
  tree match {
    case Leaf(x) => Leaf(f(x))
    case Branch(l,r) => Branch(map(l)(f), map(r)(f))
  }

defined function map

### Exercise 3.29: `fold`
reimplement `size`, `maximum`, `depth`, `map`

In [51]:
def fold[A,B](tree: Tree[A])(leaf: A => B)(branch: (B,B) => B): B =
  tree match {
    case Leaf(x) => leaf(x)
    case Branch(l,r) => branch(fold(l)(leaf)(branch), fold(r)(leaf)(branch))
  }

defined function fold

In [52]:
def size_1[A](tree: Tree[A]): Int =
  fold(tree) {_ => 1 } { (l,r) => 1+l+r }
def maximum_1(tree: Tree[Int]) =
  fold(tree) { x => x } { (l,r) => l max r }
def depth_1[A](tree: Tree[A]): Int =
  fold(tree) { _ => 1 } { (l,r) => 1 + (l max r) }
def map_1[A,B](tree: Tree[A])(f: A => B): Tree[B] =
  fold[A,Tree[B]](tree) { x => Leaf(f(x)) } { (l,r) => Branch(l,r) }

defined function size_1
defined function maximum_1
defined function depth_1
defined function map_1

# Chapter 6. State

## RNG

In [58]:
trait RNG {
  def nextInt: (Int, RNG) // Should generate a random `Int`. We'll later define other functions in terms of `nextInt`.
}

defined trait RNG

In [59]:
case class SimpleRNG(seed: Long) extends RNG {
  def nextInt: (Int, RNG) = {
    val newSeed = (seed * 0x5DEECE66DL + 0xBL) & 0xFFFFFFFFFFFFL // `&` is bitwise AND. We use the current seed to generate a new seed.
    val nextRNG = SimpleRNG(newSeed) // The next state, which is an `RNG` instance created from the new seed.
    val n = (newSeed >>> 16).toInt // `>>>` is right binary shift with zero fill. The value `n` is our new pseudo-random integer.
    (n, nextRNG) // The return value is a tuple containing both a pseudo-random integer and the next `RNG` state.
  }
}

defined class SimpleRNG

helper for testing:

In [62]:
case class ConstRNG(i: Int) extends RNG {
  def nextInt: (Int, RNG) = (i, this)
}

defined class ConstRNG

### Exercise 6.1

In [60]:
def nonNegativeInt(rng: RNG): (Int, RNG) = {
  val (i, rng1) = rng.nextInt
  val i1 =
    if (i < 0) -(i+1)
    else i
  (i1, rng1)
}

defined function nonNegativeInt

In [65]:
nonNegativeInt(ConstRNG(+2))
nonNegativeInt(ConstRNG(-2))
nonNegativeInt(SimpleRNG(1L))

res64_0: (Int, RNG) = (2, ConstRNG(2))
res64_1: (Int, RNG) = (1, ConstRNG(-2))
res64_2: (Int, RNG) = (384748, SimpleRNG(25214903928L))

### Exercise 6.2

In [66]:
def double(rng: RNG): (Double, RNG) = {
  val (i, rng1) = nonNegativeInt(rng)
  (i.toDouble / Int.MaxValue, rng1)
}

defined function double

### Exercise 6.3

In [71]:
def intDouble(rng: RNG): ((Int, Double), RNG) = {
  val (i, rng1) = rng.nextInt
  val (d, rng2) = double(rng1)
  ((i,d), rng2)
}

def doubleInt(rng: RNG): ((Double, Int), RNG) = {
  val (d, rng1) = double(rng)
  val (i, rng2) = rng1.nextInt
  ((d,i), rng2)
}

def double3(rng: RNG): ((Double, Double, Double), RNG) = {
  val (d1, rng1) = double(rng)
  val (d2, rng2) = double(rng1)
  val (d3, rng3) = double(rng2)
  ((d1,d2,d3), rng3)
}

defined function intDouble
defined function doubleInt
defined function double3

### Exercise 6.4

In [73]:
def ints(count: Int)(rng: RNG): (List[Int], RNG) = {
  @annotation.tailrec
  def fn(acc: List[Int], r: RNG, n: Int): (List[Int], RNG) =
    if (n == 0) (reverse(acc), r)
    else {
      val (x,r1) = r.nextInt
      fn(Cons(x, acc), r1, n-1)
    }

  fn(Nil, rng, count)
}

defined function ints

In [74]:
type Rand[+A] = RNG => (A, RNG)

defined type Rand

In [75]:
val int: Rand[Int] = _.nextInt

int: RNG => (Int, RNG) = ammonite.$sess.cmd74$Helper$$Lambda$2621/987597697@1baba0cb

In [76]:
def unit[A](a: A): Rand[A] = rng => (a, rng)

defined function unit

In [77]:
def map[A,B](s: Rand[A])(f: A => B): Rand[B] =
  rng => {
    val (a, rng2) = s(rng)
    (f(a), rng2)
  }

defined function map

**Problem**. `map` overwrites `map` previously defined for `List`:

In [77]:
map(List(1,2), _ + 1)

cmd77.sc:1: too many arguments (found 2, expected 1) for method map: (s: ammonite.$sess.cmd76.wrapper.cmd73.Rand[A])(f: A => B): ammonite.$sess.cmd76.wrapper.cmd73.Rand[B]
val res77 = map(List(1,2), _ + 1)
                             ^Compilation Failed

: 

## State

In [67]:
type State[S, +A] = S => (A,S)

defined type State

### Exercise 6.10

In [68]:
case class State[S,+A](run: S => (A,S)) {
  def flatMap[B](f: A => State[S,B]): State[S,B] = State(s => {
    val (a, s1) = run(s)
    f(a).run(s1)
  })
  def map[B](f: A => B): State[S, B] = flatMap { a =>
    State.unit(f(a))
  }
  def map2[B,C](state2: State[S,B])(f: (A,B) => C): State[S,C] =
    flatMap { a =>
      state2.map { f(a,_) }
    }
}

object State {
  def unit[S,A](a: A): State[S,A] = new State[S,A](s => (a,s))

  def sequence[S,A](fs: List[State[S,A]]): State[S, List[A]] =
    foldLeft(fs, unit[S,List[A]](Nil)) { (rl, ra) => ra.map2(rl) { Cons(_, _) } }

  def get[S]: State[S,S] = State[S,S](s => (s,s))
  def set[S](s: S): State[S,Unit] = State(_ => ((),s))

  def modify[S](f: S => S): State[S,Unit] = for {
    s <- get
    _ <- set(f(s))
  } yield ()
}

defined class State
defined object State

## Finite State Machine

In [69]:
object FSM {
  sealed trait Input
  case object Coin extends Input
  case object Turn extends Input
}

defined object FSM

In [70]:
case class Machine(locked: Boolean, candies: Int, coins: Int)

def simulateMachine(inputs: List[FSM.Input]): State[Machine, (Int, Int)] = {
  def step(machine: Machine, input: FSM.Input) = (machine, input) match {
    case (Machine(_, 0, _), _) => machine
    case (Machine(true, _, _), FSM.Turn) => machine
    case (Machine(false, _, _), FSM.Coin) => machine
    case (Machine(true, _, coins), FSM.Coin) => machine.copy(locked = false, coins = coins+1)
    case (Machine(false, candies, _), FSM.Turn) => machine.copy(locked = true, candies = candies-1)
  }

  State(machine => {
    val m1 = foldLeft(inputs, machine)(step)
    ((m1.coins, m1.candies), m1)
  })
}

defined class Machine
defined function simulateMachine